In [ ]:
import torch
from IPython.display import Image, clear_output  # to display images\n
from IPython.core.magic import register_line_cell_magic

print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))
torch.cuda.empty_cache()

In [7]:
# !pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
%cd ~/YOLOX
# !pip -q install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="r7Q4O1nDNtn8DK0ELj7e")
project = rf.workspace("chen-dadon-5ho55").project("pothole-d3omk")
dataset = project.version(7).download("voc")

In [ ]:
%cd ~/YOLOX
!unlink ./datasets/VOCdevkit
!ln -s {dataset.location}/train/ ./datasets/VOCdevkit

In [ ]:
%cd ~/YOLOX
%mkdir "datasets/VOCdevkit/VOC2007"
!python3 voc_txt.py "datasets/VOCdevkit/"
%mkdir "datasets/VOCdevkit/VOC2012"
!cp -r "datasets/VOCdevkit/VOC2007/." "datasets/VOCdevkit/VOC2012"

In [3]:
# from IPython.core.magic import register_line_cell_magic

# @register_line_cell_magic
# def writetemplate(line, cell):
#     with open(line, 'w+') as f:
#         f.write(cell.format(**globals()))

In [4]:
# %%writetemplate yolox/data/datasets/voc_classes.py

# VOC_CLASSES = (
#   "pothole"
# )

In [5]:
# %%writetemplate yolox/data/datasets/coco_classes.py

# COCO_CLASSES = (
#   "pothole"
# )

In [6]:
# NUM_CLASSES = 1
# !sed -i -e 's/self.num_classes = 20/self.num_classes = {NUM_CLASSES}/g' "exps/example/yolox_voc/yolox_voc_s.py"

In [ ]:
!python tools/train.py -f exps/example/yolox_voc/yolox_voc_s.py -b 32 --fp16 -o -c yolox_s.pth -l tensorboard

In [ ]:
MODEL_PATH = "./YOLOX_outputs/yolox_voc_s/best_ckpt.pth"
!python tools/eval.py -n  yolox-s -c {MODEL_PATH} -b 32 --conf 0.001 -f exps/example/yolox_voc/yolox_voc_s.py

In [ ]:
TEST_IMAGE_PATH = "/content/valid/BloodImage_00057_jpg.rf.1ee93e9ec4d76cfaddaa7df70456c376.jpg"
!python tools/demo.py image -f ./exps/example/yolox_voc/yolox_voc_s.py -c ./YOLOX_outputs/yolox_voc_s/best_ckpt.pth --path ./Pothole-7/test --conf 0.25 --nms 0.45 --tsize 640 --save_result --device cpu

# from PIL import Image
# OUTPUT_IMAGE_PATH = "/content/YOLOX/YOLOX_outputs/yolox_voc_s/vis_res/2021_08_01_19_51_59/BloodImage_00057_jpg.rf.1ee93e9ec4d76cfaddaa7df70456c376.jpg" 
# Image.open(OUTPUT_IMAGE_PATH)